In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('data_2015_to_2021_with_topics.csv')
wiki_df = pd.read_csv('data_2015_to_2021_with_context.csv')

In [16]:
# merge the two dataframes using index
df = df.merge(wiki_df, left_index=True, right_index=True)

In [17]:
df = df[['comments', 'context']]

In [18]:
# drop rows with context as empty string
df = df[df['context'] != '']
df = df[df['context'] != 'nan']

In [19]:
import ast
df['comments'] = df['comments'].apply(ast.literal_eval)

In [20]:
# filter rows that does not have topic in the comments
z = lambda x: 'topic' in x['comments'][0]

df = df[df.apply(z, axis=1)]

In [21]:
# pivot the dataframe to have one comment per row
df_new = df.explode('comments')

In [22]:
# drop rows that have comments with no topic
df_new = df_new[df_new['comments'].apply(lambda x: 'topic' in x)]

In [23]:
df_new['question'] = df_new['comments'].apply(lambda x: x['body'])
df_new['topic'] = df_new['comments'].apply(lambda x: x['topic'])

# strip whitespaces from the topic
df_new['topic'] = df_new['topic'].apply(lambda x: x.strip())

df_new = df_new[['context', 'question', 'topic']]

In [24]:
# df_new[['question', 'topic']].to_csv('qa_evaluator.csv', index=False)
qa_evaluator = df_new[['question', 'topic']].copy()

# reset the index
qa_evaluator.reset_index(drop=True, inplace=True)

# split the dataframe into train and val using pandas
train = qa_evaluator.sample(frac=0.5, random_state=42)
val = qa_evaluator.drop(train.index)

# save the train and val dataframes
train.to_csv('qt_evaluator_train.csv', index=False)
val.to_csv('qt_evaluator_val.csv', index=False)

In [25]:
df_new['text'] = df_new.apply(lambda x: f'<topic> {x["topic"]} <context> {x["context"]}', axis=1)

In [26]:
# df_new[['question', 'text']].to_csv('question_generator.csv', index=False)
question_generator = df_new[['question', 'text']].copy()

# reset the index
question_generator.reset_index(drop=True, inplace=True)

# split the dataframe into train and val using pandas
train = question_generator.sample(frac=0.5, random_state=42)
val = question_generator.drop(train.index)

# save the train and val dataframes
train.to_csv('question_generator_train.csv', index=False)
val.to_csv('question_generator_val.csv', index=False)